In [3]:
!pip install transformers[torch]

import pandas as pd
import numpy as np
import random
import torch
import torch.nn.functional as F
import csv

from google.colab import drive
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup, pipeline, set_seed
from tqdm import tqdm, trange

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.8 MB/s eta 0:00:00


#Class/Def codeblocks

In [4]:
class PoemsDataset(Dataset):
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.poems = []

        for row in df['Poems']:
          self.poems.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))
        if truncate:
            self.poems = self.poems[:20000]
        self.poems_count = len(self.poems)

    def __len__(self):
        return self.poems_count

    def __getitem__(self, item):
        return self.poems[item]

In [5]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [6]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,):

    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [8]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=1, ####
    entry_length=50, #####maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break

            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>"
              generated_list.append(output_text)

    return generated_list

In [9]:
#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_poems = []
  for i in range(len(test_data)):
    if test_data['Poems'][i]:
      x = generate(model, tokenizer, test_data['Poems'][i], entry_count=1)
    generated_poems.append(x)
  return generated_poems

#Generate model

In [10]:
#Connect database
drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/kaggle_poem_dataset.csv'

#pre-process
df = pd.read_csv(dataset_path,index_col=0)
df = df.drop(columns=['Title','Author','Poetry Foundation ID'])
df['Content']= df['Content'].str.replace('\n',' ')
df = df[df['Content'].apply(lambda x: len(x.split(' ')) < 350)]
df = df.rename(columns={'Content':'Poems'})
df = df.dropna()


Mounted at /content/drive


In [ ]:
dataset = PoemsDataset(df['Poems'], truncate=True, gpt2_type="gpt2")

#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

model = train(dataset, model, tokenizer)
torch.save(model, '/content/drive/MyDrive/checkpoint_whole.pth')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


12967it [16:51, 12.81it/s]


Training epoch 1
tensor(1.4783, device='cuda:0', grad_fn=<NllLossBackward0>)


12967it [16:58, 12.73it/s]


Training epoch 2
tensor(1.9874, device='cuda:0', grad_fn=<NllLossBackward0>)


10403it [13:30, 13.36it/s]

#generate poems

In [ ]:
model = torch.load('/content/drive/MyDrive/checkpoint_whole.pth')

In [ ]:
text = "The most lovely poem about winter is writtem as follows:"

In [ ]:
x = generate(model.to('cpu'), tokenizer, text, entry_count=1)
poem_string = x[0]

# Splitting the poem using periods and commas
poem_lines = [line.strip() for line in poem_string.split('.') if line]
poem_lines = [line + '.' for line in poem_lines]

formatted_lines = []
for line in poem_lines:
    formatted_lines.extend([subline.strip() for subline in line.split('\n') if subline])

for line in formatted_lines:
    print(line)